In [ ]:
import os
import cwt
import numpy as np
from cwt import auth

url = os.environ.get('WPS_URL', 'https://aims2.llnl.gov/wps')
token = os.environ.get('WPS_TOKEN', None)
verify = os.environ.get('WPS_VERIFY', True)
timeout = os.environ.get('WPS_TIMEOUT', 120)

In [ ]:
token = auth.TokenAuthenticator(token, key='COMPUTE_TOKEN', value='{}')

client = cwt.WPSClient(url, auth=token, verify=verify)

In [ ]:
d0 = cwt.Domain(time=('2008-01-02T09:00:00.000000000', '2008-01-03T15:00:00.000000000'), lat=(-45, 45))

v0 = cwt.Variable('http://aims3.llnl.gov/thredds/dodsC/cmip5_css01_data/cmip5/output1/CNRM-CERFACS/CNRM-CM5/amip/3hr/atmos/cf3hr/r1i1p1/v20120203/pr/pr_cf3hr_CNRM-CM5_amip_r1i1p1_200801010300-200901010000.nc', 'pr')

In [ ]:
subset = client.CDAT.subset(v0, domain=d0)

scaled = client.CDAT.multiply(subset, const=10, variable='pr')

binned = client.CDAT.groupby_bins(scaled, bins=np.arange(0, 0.02391861, 0.001).tolist(), variable='pr')

mean = client.CDAT.mean(binned, rename=['pr', 'pr_mean'])

std = client.CDAT.std(binned, rename=['pr', 'pr_std'])

var = client.CDAT.var(binned, rename=['pr', 'pr_var'])

workflow = client.CDAT.workflow(mean, std, var)

In [ ]:
client.execute(workflow)

workflow.wait(timeout=timeout)

In [ ]:
import xarray as xr
import hashlib

ds = [xr.open_dataset(x.uri) for x in workflow.output]

def find_name(ds):
    for x in ('pr_mean', 'pr_std', 'pr_var'):
        if x in ds:
            return x
    return None

var_names = [find_name(x) for x in ds]

digest = dict((x, hashlib.sha256(y[x].values.tobytes()).hexdigest()) for x, y in zip(var_names, ds))

assert digest['pr_var'] == 'e7c3b44b61d87a9e0b188fda59f8df2345cce94a9d7cd40d34743b10fffd61f2'
assert digest['pr_std'] == '9fd459280dd7601cbf716ec17428e8e6e686171e3a1195594d8c51d4e8751ed5'
assert digest['pr_mean'] == '4bb4b0a19cf52c748f1892e825373631ebf8e9629bde2e69ca0d941af80b01fc'